# Generate Dataset
using the raw data in database, we will generate dataset using DatasetGenerator

In [1]:
from datetime import datetime

from dataset_generator import DatasetGenerator

In [4]:
dataset_generator = DatasetGenerator(start_date=datetime(2019, 1, 1), end_date=datetime(2019, 1, 2))
X, Y = dataset_generator.generate_dataset()

generate dataset...


3109598it [01:01, 50967.09it/s] 

generate dataset done.


In [6]:
X.shape, Y.shape

((2556348, 3), (2556348,))